# 随机森林 (Random Forest)

## 算法原理

随机森林是 Bagging 的扩展，在决策树的基础上引入了额外的随机性：

1. **样本随机**: 使用 Bootstrap 有放回抽样创建不同的训练子集
2. **特征随机**: 在每个节点分裂时，只考虑随机选择的特征子集

这两层随机性使得各棵树更加独立，从而更有效地降低集成的方差。

## 核心特点

| 特性 | 说明 |
|------|------|
| 集成方式 | Bagging（并行集成） |
| 基学习器 | 决策树 |
| 样本采样 | Bootstrap（有放回抽样） |
| 特征采样 | 每个节点随机选择 $\sqrt{p}$ 个特征（分类）或 $p/3$ 个（回归） |
| 聚合方式 | 投票（分类）/ 平均（回归） |

## 优势

- 几乎不需要特征缩放
- 对异常值和缺失值相对鲁棒
- 提供特征重要性估计
- 支持 OOB（Out-of-Bag）评估

In [ ]:
# 导入必要的库
import numpy as np
from sklearn.datasets import make_moons
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report

# 设置随机种子
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

## 1. 数据准备

In [ ]:
# 生成月牙形数据集
X, y = make_moons(n_samples=1000, noise=0.30, random_state=RANDOM_STATE)

# 划分数据集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE
)

print(f"训练集大小: {X_train.shape[0]}")
print(f"测试集大小: {X_test.shape[0]}")
print(f"特征维度: {X.shape[1]}")

## 2. 使用 RandomForestClassifier

sklearn 提供了专门的 RandomForestClassifier 实现。

In [ ]:
# 创建随机森林分类器
rf_clf = RandomForestClassifier(
    n_estimators=500,          # 树的数量
    max_leaf_nodes=16,         # 每棵树的最大叶子节点数
    max_features='sqrt',       # 每个节点考虑的特征数（分类问题常用 sqrt）
    bootstrap=True,            # 使用 bootstrap 抽样
    oob_score=True,            # 计算 OOB 分数
    n_jobs=-1,                 # 使用所有 CPU 核心
    random_state=RANDOM_STATE
)

# 训练模型
rf_clf.fit(X_train, y_train)

# 评估
y_pred = rf_clf.predict(X_test)
rf_accuracy = accuracy_score(y_test, y_pred)

print(f"测试集准确率: {rf_accuracy:.4f}")
print(f"OOB 分数: {rf_clf.oob_score_:.4f}")

## 3. 使用 BaggingClassifier 实现随机森林

随机森林也可以用 BaggingClassifier + 随机特征决策树来实现。

In [ ]:
# 使用 BaggingClassifier 实现类似随机森林的效果
# 通过设置 splitter='random' 在决策树中引入随机性
bag_clf = BaggingClassifier(
    estimator=DecisionTreeClassifier(
        splitter='random',       # 随机选择最佳分割
        max_leaf_nodes=16,
        random_state=RANDOM_STATE
    ),
    n_estimators=500,
    max_samples=1.0,             # 使用全部样本（bootstrap 会自动抽样）
    bootstrap=True,
    n_jobs=-1,
    random_state=RANDOM_STATE
)

# 训练模型
bag_clf.fit(X_train, y_train)

# 评估
bag_accuracy = bag_clf.score(X_test, y_test)

print(f"BaggingClassifier 准确率: {bag_accuracy:.4f}")

## 4. 与单棵决策树对比

比较随机森林与单棵决策树的性能差异。

In [ ]:
# 单棵决策树
tree_clf = DecisionTreeClassifier(
    max_leaf_nodes=16,
    random_state=RANDOM_STATE
)
tree_clf.fit(X_train, y_train)
tree_accuracy = tree_clf.score(X_test, y_test)

# 对比结果
print("模型性能对比:")
print(f"{'模型':<25} {'准确率':>10}")
print("-" * 37)
print(f"{'单棵决策树':<25} {tree_accuracy:>10.4f}")
print(f"{'RandomForestClassifier':<25} {rf_accuracy:>10.4f}")
print(f"{'BaggingClassifier':<25} {bag_accuracy:>10.4f}")

## 5. 交叉验证评估

In [ ]:
# 交叉验证
cv_scores = cross_val_score(rf_clf, X, y, cv=5, scoring='accuracy')

print(f"交叉验证分数: {cv_scores}")
print(f"平均准确率: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

## 6. 特征重要性

随机森林可以自动计算特征重要性，基于每个特征在所有树中的不纯度减少量。

In [ ]:
# 获取特征重要性
feature_names = ['Feature_1', 'Feature_2']
importances = rf_clf.feature_importances_

print("特征重要性:")
for name, importance in zip(feature_names, importances):
    print(f"  {name}: {importance:.4f}")

## 7. 参数敏感性分析

In [ ]:
# 分析树的数量对性能的影响
n_estimators_range = [10, 50, 100, 200, 500]

print("树的数量 vs 性能:")
for n_est in n_estimators_range:
    clf = RandomForestClassifier(
        n_estimators=n_est,
        max_leaf_nodes=16,
        n_jobs=-1,
        random_state=RANDOM_STATE
    )
    clf.fit(X_train, y_train)
    accuracy = clf.score(X_test, y_test)
    print(f"  n_estimators={n_est:3d}: 准确率={accuracy:.4f}")

## 8. 单元测试验证

In [ ]:
def test_random_forest():
    """随机森林功能测试"""
    
    # 测试1: 模型应该正确训练
    assert hasattr(rf_clf, 'estimators_'), "模型未正确训练"
    
    # 测试2: 树的数量应该正确
    assert len(rf_clf.estimators_) == 500, "树的数量不正确"
    
    # 测试3: OOB 分数应该存在且在合理范围
    assert hasattr(rf_clf, 'oob_score_'), "OOB 分数未计算"
    assert 0.7 <= rf_clf.oob_score_ <= 1.0, "OOB 分数不在合理范围"
    
    # 测试4: 随机森林应该比单棵树表现更好或相当
    assert rf_accuracy >= tree_accuracy * 0.95, "随机森林性能异常低"
    
    # 测试5: 特征重要性应该存在
    assert hasattr(rf_clf, 'feature_importances_'), "特征重要性未计算"
    assert abs(sum(importances) - 1.0) < 0.01, "特征重要性之和应接近1"
    
    # 测试6: 预测结果形状应该正确
    assert y_pred.shape == y_test.shape, "预测结果形状不正确"
    
    print("所有测试通过!")

test_random_forest()

## 总结

### 随机森林的优势

1. **开箱即用**: 默认参数通常就能获得不错的性能
2. **并行训练**: 各棵树独立训练，可充分利用多核 CPU
3. **特征重要性**: 自动计算特征重要性
4. **OOB 评估**: 无需额外验证集即可评估性能

### 调参建议

- `n_estimators`: 越多越好，但收益递减，通常 100-500
- `max_features`: 分类问题用 `sqrt`，回归问题用 `log2` 或全部特征
- `max_depth`: 默认不限制，可适当限制以减少过拟合
- `min_samples_leaf`: 增加可以减少过拟合

### 与其他方法的对比

- **vs 单棵树**: 更稳定，方差更低
- **vs Bagging**: 额外的特征随机性进一步降低方差
- **vs Boosting**: 训练更快，但精度可能略低